In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Helper Function for Text Cleaning:

#Build a Text Cleaning Pipeline

In [2]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Make sure all required NLTK data is downloaded
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def text_cleaning_pipeline(text, rule="lemmatize"):
    """
    This function cleans a given text by:
    - Lowercasing
    - Removing URLs
    - Removing emojis
    - Removing unwanted characters (punctuation, numbers, etc.)
    - Tokenizing
    - Removing stopwords
    - Applying lemmatization or stemming
    """

    # Convert to lowercase
    data = text.lower()

    # Remove URLs
    data = re.sub(r'http\S+|www\S+|https\S+', '', data, flags=re.MULTILINE)

    # Remove emojis (basic)
    data = re.sub(r'[^\x00-\x7F]+', '', data)

    # Remove punctuation and numbers
    data = re.sub(r'[^a-z\s]', '', data)
    data = re.sub("#[A-Za-z0-9_]+","", data)
    data = re.sub("@[A-Za-z0-9_]+"," ", data)

    # Tokenize
    tokens = nltk.word_tokenize(data)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize or stem
    if rule == "lemmatize":
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    elif rule == "stem":
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
    else:
        print("Pick between 'lemmatize' or 'stem'")
        return ""

    return " ".join(tokens)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


#Text Classification using Machine Learning Models

Loading the dataset

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/AI and ML Workshop/Week-8/trum_tweet_sentiment_analysis.csv')
df.head()

,text,Sentiment
0,RT @JohnLeguizamo: #trump not draining swamp b...,0
1,ICYMI: Hackers Rig FM Radio Stations To Play A...,0
2,Trump protests: LGBTQ rally in New York https:...,1
3,"""Hi I'm Piers Morgan. David Beckham is awful b...",0
4,RT @GlennFranco68: Tech Firm Suing BuzzFeed fo...,0


Text Cleaning and Tokenization

In [4]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Make sure all required NLTK data is downloaded
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def text_cleaning_pipeline(text, rule="lemmatize"):
    """
    This function cleans a given text by:
    - Lowercasing
    - Removing URLs
    - Removing emojis
    - Removing unwanted characters (punctuation, numbers, etc.)
    - Tokenizing
    - Removing stopwords
    - Applying lemmatization or stemming
    """
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove emojis (basic)
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Remove punctuation and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    text = re.sub("@[A-Za-z0-9_]+"," ", text)

    # Tokenize
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize or stem
    if rule == "lemmatize":
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    elif rule == "stem":
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
    else:
        print("Pick between 'lemmatize' or 'stem'")
        return ""

    return " ".join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
df["cleaned_text"] = df["text"].apply(text_cleaning_pipeline)
df[["text", "cleaned_text"]].head()

,text,cleaned_text
0,RT @JohnLeguizamo: #trump not draining swamp b...,rt johnleguizamo trump draining swamp taxpayer...
1,ICYMI: Hackers Rig FM Radio Stations To Play A...,icymi hacker rig fm radio station play antitru...
2,Trump protests: LGBTQ rally in New York https:...,trump protest lgbtq rally new york bbcworld vi...
3,"""Hi I'm Piers Morgan. David Beckham is awful b...",hi im pier morgan david beckham awful donald t...
4,RT @GlennFranco68: Tech Firm Suing BuzzFeed fo...,rt glennfranco tech firm suing buzzfeed publis...


Train-Test Split

In [7]:
from sklearn.model_selection import train_test_split

X = df["cleaned_text"]
y = df["Sentiment"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

TF-IDF Vectorization

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


Model Training & Evaluation

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(max_iter=300)
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94    248563
           1       0.90      0.85      0.88    121462

    accuracy                           0.92    370025
   macro avg       0.91      0.90      0.91    370025
weighted avg       0.92      0.92      0.92    370025

